**PARAMS**

In [35]:
model_filename = '/content/drive/MyDrive/HomeworkANN/Homework2/NetOutput/U-NET/Nico/best/56_BottleNeck_globalContext_Unet++.keras'

#timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_"+"BottleNeck_globalContext_Unet++_CRF.csv"

print(f"Model filename: {model_filename}")
print(f"Submission filename: {submission_filename}")

Model filename: /content/drive/MyDrive/HomeworkANN/Homework2/NetOutput/U-NET/Nico/best/56_BottleNeck_globalContext_Unet++.keras
Submission filename: submission_BottleNeck_globalContext_Unet++_CRF.csv


#Generic imports

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import os
from datetime import datetime

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

from tensorflow import keras


import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tfk.__version__}")
print(f"GPU devices: {len(tf.config.list_physical_devices('GPU'))}")

TensorFlow version: 2.17.1
Keras version: 3.5.0
GPU devices: 1


#Import Model and TestSet

In [28]:
prepath = '/content/drive/MyDrive/HomeworkANN/Homework2/Dataset/InitialDataset'

In [29]:
import numpy as np

filename = '/mars_for_students.npz'

complete_path = prepath+filename

data = np.load(complete_path)
training_set = data["training_set"]

X_train = training_set[:, 0]
y_train = training_set[:, 1]

X_test = data["test_set"]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)


(2615, 64, 128)
(2615, 64, 128)
(10022, 64, 128)


In [30]:

class MeanIntersectionOverUnion(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes, labels_to_exclude=None, name="mean_iou", dtype=None):
        super(MeanIntersectionOverUnion, self).__init__(num_classes=num_classes, name=name, dtype=dtype)
        if labels_to_exclude is None:
            labels_to_exclude = [0]  # Default to excluding label 0
        self.labels_to_exclude = labels_to_exclude

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert predictions to class labels
        y_pred = tf.math.argmax(y_pred, axis=-1)

        # Flatten the tensors
        y_true = tf.reshape(y_true, [-1])
        y_pred = tf.reshape(y_pred, [-1])

        # Apply mask to exclude specified labels
        for label in self.labels_to_exclude:
            mask = tf.not_equal(y_true, label)
            y_true = tf.boolean_mask(y_true, mask)
            y_pred = tf.boolean_mask(y_pred, mask)

        # Update the state
        return super().update_state(y_true, y_pred, sample_weight)


In [31]:
from google.colab import files

model = tfk.models.load_model(
    model_filename, compile=False
)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=["accuracy", MeanIntersectionOverUnion(num_classes=5, labels_to_exclude=[0])]
)

print(f"Model loaded from {model_filename}")

Model loaded from /content/drive/MyDrive/HomeworkANN/Homework2/NetOutput/U-NET/Nico/best/56_BottleNeck_globalContext_Unet++.keras


#CRF

In [32]:
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git


  Cloning https://github.com/lucasb-eyer/pydensecrf.git to /tmp/pip-req-build-k8k7lrln
  Running command git clone --filter=blob:none --quiet https://github.com/lucasb-eyer/pydensecrf.git /tmp/pip-req-build-k8k7lrln
  Resolved https://github.com/lucasb-eyer/pydensecrf.git to commit 2723c7fa4f2ead16ae1ce3d8afe977724bb8f87f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [33]:
# CRF library imports
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax, create_pairwise_gaussian, create_pairwise_bilateral

# Standard imports
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import files

In [34]:
def apply_crf(image, probs, num_classes=5):
    """
    Applies CRF post-processing to refine segmentation results.
    Args:
        image: Input image (grayscale or RGB).
        probs: Softmax probabilities (H, W, num_classes).
        num_classes: Number of segmentation classes.
    Returns:
        Refined segmentation mask.
    """
    h, w, _ = probs.shape

    # Ensure probabilities array is contiguous
    probs = probs.copy()

    crf = dcrf.DenseCRF2D(w, h, num_classes)

    # Create unary potentials from the predicted probabilities
    unary = unary_from_softmax(probs.transpose(2, 0, 1).copy())
    crf.setUnaryEnergy(unary)

    # Add pairwise Gaussian potential (spatial consistency)
    gaussian_energy = create_pairwise_gaussian(sdims=(3, 3), shape=(h, w))
    crf.addPairwiseEnergy(gaussian_energy, compat=3)

    # Add pairwise bilateral potential (appearance consistency)
    bilateral_energy = create_pairwise_bilateral(
        sdims=(50, 50), schan=(20,), img=image, chdim=-1
    )
    crf.addPairwiseEnergy(bilateral_energy, compat=10)

    # Perform CRF inference
    refined_labels = crf.inference(5)
    refined_labels = np.argmax(refined_labels, axis=0).reshape(h, w)

    return refined_labels


In [36]:
# Predict softmax probabilities
preds_probs = model.predict(X_test)
preds_probs = tf.nn.softmax(preds_probs, axis=-1).numpy()

# Apply CRF to refine predictions
refined_preds = []
for i in range(X_test.shape[0]):
    # Assuming X_test is grayscale; adjust for channels if needed
    image = np.squeeze(X_test[i]).copy()  # Ensure image is contiguous
    probs = preds_probs[i].copy()  # Ensure probabilities are contiguous
    refined_mask = apply_crf(image, probs)  # Apply CRF
    refined_preds.append(refined_mask)

# Convert to numpy array
refined_preds = np.array(refined_preds)
print(f"Refined predictions shape: {refined_preds.shape}")


314/314 ━━━━━━━━━━━━━━━━━━━━ 17s 49ms/step
Refined predictions shape: (10022, 64, 128)


#Create submission file

In [37]:
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

# Create submission DataFrame
submission_df = y_to_df(refined_preds)

# Save submission file
submission_df.to_csv(submission_filename, index=False)
files.download(submission_filename)

print(f"Submission file {submission_filename} created with CRF post-processing.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submission file submission_BottleNeck_globalContext_Unet++_CRF.csv created with CRF post-processing.
